In [10]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import seaborn as sns
from sklearn.externals.six import StringIO
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
import pydotplus
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from ipywidgets import *
from IPython.display import display
from IPython.display import Image
plt.rcParams["figure.figsize"] = (20, 15)
plt.rcParams["font.size"] = 14
plt.style.use("ggplot")
sns.set(font_scale=1.5)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, pacf
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
# from pygeocoder import Geocoder
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from pandas.api.types import CategoricalDtype
from sklearn.linear_model import LinearRegression,LogisticRegression, Ridge, Lasso, RidgeCV, LassoCV
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import r2_score, mean_squared_error
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
std_scaler = StandardScaler()
rob_scaler = RobustScaler()
mm_scaler = MinMaxScaler()

# DATA

In [2]:
austin = pd.read_pickle("../pickled_data/austin_all.pickled")
sanfran = pd.read_pickle("../pickled_data/sf_all_df.pickled")
LA = pd.read_pickle("../pickled_data/LA_new_df.pickled")
chicago = pd.read_pickle("../pickled_data/chicago_trips_df.pickled")
london = pd.read_pickle("../pickled_data/london_counts_df.pickled")
DC = pd.read_pickle("../pickled_data/DC_df.pickled")

# PIPELINE

In [11]:
scaler = StandardScaler()
# model = LogisticRegression(solver='lbfgs', random_state=1)
model = LogisticRegressionCV(solver='lbfgs', multi_class='ovr', cv=5)
log_params = {'penalty': ['l1', 'l2'],
                   'solver': ['liblinear'],
                   'Cs': [np.logspace(3, 0, 10)]}
grid_log = GridSearchCV(model, log_params, cv=5, n_jobs=2)
pipe = Pipeline(steps=[('scaler', scaler),
                       ('model', grid_log)])


classifier = DecisionTreeClassifier(criterion='gini',
                                    max_depth=3, 
                                    random_state=1)
g_params = {'max_depth':list(range(1, 11))+[None],'max_features':[None, 1, 2, 3],'max_leaf_nodes':[None,2,3,4],
             'min_samples_leaf':[2,3,4, 5, 10, 15, 20, 25, 30, 40, 50],'min_samples_split':[2,3]}
gridsearch = GridSearchCV(classifier , g_params ,return_train_score=True,  cv=10)
pipe_class = Pipeline(steps=[('scaler', scaler),
                       ('model', gridsearch)])

In [23]:
X_train

,duration,trip_id
start_time,,
2018-07-12,41.589286,9.675733e+07
2018-05-04,33.869352,8.518074e+07
2018-03-21,23.051230,7.797141e+07
2018-12-29,54.303937,1.123848e+08
2019-06-09,37.780872,1.232019e+08
...,...,...
2019-05-17,41.827688,1.216759e+08
2019-01-07,31.076132,1.129813e+08
2017-11-09,31.836265,5.711803e+07


# AUSTIN

In [12]:
# austin_per_day = austin.groupby("dateasdatetime").mean()
# austin_per_day["count"] = austin.dateasdatetime.value_counts()
# both work!
austin_per_day = austin.groupby([austin["start_time"].dt.date]).mean()
austin_per_day["count"] = austin.groupby([austin["start_time"].dt.date]).count()["bikeid"]
q1 = austin_per_day['count'].quantile(q=1/3)
q2 = austin_per_day['count'].quantile(q=2/3)
q3 = austin_per_day['count'].quantile(q=3/3)
list_bin = []
for val in austin_per_day['count']:
    if val < q1:
        list_bin.append(0)
    elif val>= q1 and val <q2:
        list_bin.append(1)
    else:
        list_bin.append(2)
        
austin_per_day['count_class'] = list_bin
austin_per_day.dropna(inplace=True)
del austin_per_day['count']

X = austin_per_day.copy()
y = X.pop('count_class')
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=1)
pipe.fit(X_train, y_train)
pipe_class.fit(X_train, y_train)
print('AUSTIN:')
print('--------------------------')
print('Baseline:',austin_per_day.count_class.value_counts(normalize=True).max())
print('--------------------------')
print('Logistic Regression')
print('--------------------------')
print('Cross validation' , cross_val_score(pipe, X_train, y_train, cv=5).mean())
print('train_score:',pipe.score(X_train, y_train))
print('test_score:',pipe.score(X_test, y_test))
print('--------------------------')
print('Grid search: Decision Tree Classifier')
print('--------------------------')
print('Cross validation' , cross_val_score(pipe_class, X_train, y_train, cv=5).mean())
print('train_score:',pipe_class.score(X_train, y_train))
print('test_score:',pipe_class.score(X_test, y_test))

AUSTIN:
--------------------------
Baseline: 0.3344
--------------------------
Logistic Regression
--------------------------
Cross validation 0.6125286609941781
train_score: 0.6308571428571429
test_score: 0.6026666666666667
--------------------------
Grid search: Decision Tree Classifier
--------------------------
Cross validation 0.5931974175250037
train_score: 0.664
test_score: 0.5706666666666667


#  SF

In [13]:
sf_per_day = sanfran.groupby([sanfran["start_date"].dt.date]).mean()
sf_per_day["count"] = sanfran.groupby([sanfran["start_date"].dt.date]).count()["mean_temperature_c"]
del sf_per_day['mean_humidity_binary']
q1 = sf_per_day['count'].quantile(q=1/3)
q2 = sf_per_day['count'].quantile(q=2/3)
q3 = sf_per_day['count'].quantile(q=3/3)
list_bin = []
for val in sf_per_day['count']:
    if val < q1:
        list_bin.append(0)
    elif val>= q1 and val < q2:
        list_bin.append(1)
    else:
        list_bin.append(2)
        
sf_per_day['count_class'] = list_bin
sf_per_day.dropna(inplace=True)
del sf_per_day['count']

X = sf_per_day.copy()
y = X.pop('count_class')
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=1)
pipe.fit(X_train, y_train)
pipe_class.fit(X_train, y_train)
print('SAN FRANCISCO:')
print('--------------------------')
print('Baseline:',sf_per_day.count_class.value_counts(normalize=True).max())
print('--------------------------')
print('Logistic Regression')
print('--------------------------')
print('Cross validation' , cross_val_score(pipe, X_train, y_train, cv=5).mean())
print('train_score:',pipe.score(X_train, y_train))
print('test_score:',pipe.score(X_test, y_test))
print('--------------------------')
print('Grid search: Decision Tree Classifier')
print('--------------------------')
print('Cross validation' , cross_val_score(pipe_class, X_train, y_train, cv=5).mean())
print('train_score:',pipe_class.score(X_train, y_train))
print('test_score:',pipe_class.score(X_test, y_test))

SAN FRANCISCO:
--------------------------
Baseline: 0.33424283765347884
--------------------------
Logistic Regression
--------------------------
Cross validation 0.8657703081232493
train_score: 0.8752436647173489
test_score: 0.8363636363636363
--------------------------
Grid search: Decision Tree Classifier
--------------------------
Cross validation 0.8323809523809524
train_score: 0.9044834307992202
test_score: 0.8272727272727273


# LA

In [26]:
LA_per_day = LA.groupby([LA["start_time"].dt.date]).mean()
LA_per_day["count"] = LA.groupby([LA["start_time"].dt.date]).count()["bike_id"]
q1 = LA_per_day['count'].quantile(q=1/3)
q2 = LA_per_day['count'].quantile(q=2/3)
q3 = LA_per_day['count'].quantile(q=3/3)
list_bin = []
for val in LA_per_day['count']:
    if val < q1:
        list_bin.append(0)
    elif val>= q1 and val <q2:
        list_bin.append(1)
    else:
        list_bin.append(2)
        
LA_per_day['count_class'] = list_bin
LA_per_day.dropna(inplace=True)
del LA_per_day['count']

X = LA_per_day.copy()
y = X.pop('count_class')
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=1)
pipe.fit(X_train, y_train)

print('LA:')
print('--------------------------')
print('Baseline:',sf_per_day.count_class.value_counts(normalize=True).max())
print('--------------------------')
print('Logistic Regression')
print('--------------------------')
print('Cross validation' , cross_val_score(pipe, X_train, y_train, cv=5).mean())
print('train_score:',pipe.score(X_train, y_train))
print('test_score:',pipe.score(X_test, y_test))
print('--------------------------')

classifier_1 = DecisionTreeClassifier(criterion='gini',
                                    max_depth=3, 
                                    random_state=1)
g_params_1 = {'max_depth':list(range(1, 11))+[None],'max_features':[1, 2],'max_leaf_nodes':[None,2,3,4],
             'min_samples_leaf':[2,3,4, 5, 10, 15, 20, 25, 30, 40, 50],'min_samples_split':[2,3]}
gridsearch_1 = GridSearchCV(classifier_1 , g_params_1 ,return_train_score=True,  cv=10)
pipe_class_1 = Pipeline(steps=[('scaler', scaler),
                       ('model', gridsearch_1)])
pipe_class_1.fit(X_train, y_train)
print('Grid search: Decision Tree Classifier')
print('--------------------------')
print('Cross validation' , cross_val_score(pipe_class_1, X_train, y_train, cv=5).mean())
print('train_score:',pipe_class_1.score(X_train, y_train))
print('test_score:',pipe_class_1.score(X_test, y_test))

LA:
--------------------------
Baseline: 0.33424283765347884
--------------------------
Logistic Regression
--------------------------
Cross validation 0.5328685136996786
train_score: 0.5669515669515669
test_score: 0.5980066445182725
--------------------------
Grid search: Decision Tree Classifier
--------------------------
Cross validation 0.6495476445591247
train_score: 0.7450142450142451
test_score: 0.7242524916943521


# Chicago

In [16]:
chicago_per_day = chicago.groupby([chicago["start_time"].dt.date]).mean()
chicago_per_day["count"] = chicago.groupby([chicago["start_time"].dt.date]).count()["trip_id"]
q1 = chicago_per_day['count'].quantile(q=1/3)
q2 = chicago_per_day['count'].quantile(q=2/3)
q3 = chicago_per_day['count'].quantile(q=3/3)
list_bin = []
for val in chicago_per_day['count']:
    if val < q1:
        list_bin.append(0)
    elif val>= q1 and val <q2:
        list_bin.append(1)
    else:
        list_bin.append(2)
        
chicago_per_day['count_class'] = list_bin
chicago_per_day.dropna(inplace=True)
del chicago_per_day['count']

X = chicago_per_day.copy()
y = X.pop('count_class')

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=1)
pipe.fit(X_train, y_train)
pipe_class.fit(X_train, y_train)
print('CHICAGO:')
print('--------------------------')
print('Baseline:',sf_per_day.count_class.value_counts(normalize=True).max())
print('--------------------------')
print('Logistic Regression')
print('--------------------------')
print('Cross validation' , cross_val_score(pipe, X_train, y_train, cv=5).mean())
print('train_score:',pipe.score(X_train, y_train))
print('test_score:',pipe.score(X_test, y_test))
print('--------------------------')
print('Grid search: Decision Tree Classifier')
print('--------------------------')
print('Cross validation' , cross_val_score(pipe_class, X_train, y_train, cv=5).mean())
print('train_score:',pipe_class.score(X_train, y_train))
print('test_score:',pipe_class.score(X_test, y_test))

CHICAGO:
--------------------------
Baseline: 0.33424283765347884
--------------------------
Logistic Regression
--------------------------
Cross validation 0.6431372549019608
train_score: 0.6666666666666666
test_score: 0.7181818181818181
--------------------------
Grid search: Decision Tree Classifier
--------------------------
Cross validation 0.7176470588235294
train_score: 0.792156862745098
test_score: 0.7636363636363637


# LONDON

In [17]:
london_per_day = london.groupby([london["timestamp"].dt.date]).mean()
london_per_day.drop(['is_weekend','hum_binary','t2'],axis=1,inplace=True)
q1 = london_per_day['cnt'].quantile(q=1/3)
q2 = london_per_day['cnt'].quantile(q=2/3)
q3 = london_per_day['cnt'].quantile(q=3/3)
list_bin = []
for val in london_per_day['cnt']:
    if val < q1:
        list_bin.append(0)
    elif val>= q1 and val <q2:
        list_bin.append(1)
    else:
        list_bin.append(2)
        
london_per_day['count_class'] = list_bin
del london_per_day['cnt']

X = london_per_day.copy()
y = X.pop('count_class')

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=1)
pipe.fit(X_train, y_train)
pipe_class.fit(X_train, y_train)
print('LONDON:')
print('--------------------------')
print('Baseline:',london_per_day.count_class.value_counts(normalize=True).max())
print('--------------------------')
print('Logistic Regression')
print('--------------------------')
print('Cross validation' , cross_val_score(pipe, X_train, y_train, cv=5).mean())
print('train_score:',pipe.score(X_train, y_train))
print('test_score:',pipe.score(X_test, y_test))
print('--------------------------')
print('Grid search: Decision Tree Classifier')
print('--------------------------')
print('Cross validation' , cross_val_score(pipe_class, X_train, y_train, cv=5).mean())
print('train_score:',pipe_class.score(X_train, y_train))
print('test_score:',pipe_class.score(X_test, y_test))

LONDON:
--------------------------
Baseline: 0.33424657534246577
--------------------------
Logistic Regression
--------------------------
Cross validation 0.7455549209975253
train_score: 0.7710371819960861
test_score: 0.7808219178082192
--------------------------
Grid search: Decision Tree Classifier
--------------------------
Cross validation 0.6672758423757853
train_score: 0.7729941291585127
test_score: 0.6894977168949772


#  DC

In [18]:
dc_per_day = DC.groupby([DC["datetime"].dt.date]).mean()
dc_per_day["count"] = (DC["count"].groupby([DC["datetime"].dt.date]).sum())
q1 = dc_per_day['count'].quantile(q=1/3)
q2 = dc_per_day['count'].quantile(q=2/3)
q3 = dc_per_day['count'].quantile(q=3/3)
list_bin = []
for val in dc_per_day['count']:
    if val < q1:
        list_bin.append(0)
    elif val>= q1 and val < q2:
        list_bin.append(1)
    else:
        list_bin.append(2)
                
dc_per_day['count_class'] = list_bin
del dc_per_day['count']


X = dc_per_day.copy()
y = X.pop('count_class')

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=1)
pipe.fit(X_train, y_train)
pipe_class.fit(X_train, y_train)
print('DC:')
print('--------------------------')
print('Baseline:',dc_per_day.count_class.value_counts(normalize=True).max())
print('--------------------------')
print('Logistic Regression')
print('--------------------------')
print('Cross validation' , cross_val_score(pipe, X_train, y_train, cv=5).mean())
print('train_score:',pipe.score(X_train, y_train))
print('test_score:',pipe.score(X_test, y_test))
print('--------------------------')
print('Grid search: Decision Tree Classifier')
print('--------------------------')
print('Cross validation' , cross_val_score(pipe_class, X_train, y_train, cv=5).mean())
print('train_score:',pipe_class.score(X_train, y_train))
print('test_score:',pipe_class.score(X_test, y_test))

DC:
--------------------------
Baseline: 0.3333333333333333
--------------------------
Logistic Regression
--------------------------
Cross validation 0.9306412337662338
train_score: 0.9623824451410659
test_score: 0.927007299270073
--------------------------
Grid search: Decision Tree Classifier
--------------------------
Cross validation 0.8963564213564214
train_score: 0.9843260188087775
test_score: 0.8978102189781022


In [20]:
print(pipe.predict(X_test))
print(pipe_class.predict(X_test))
#in a dataframe > value counts or uniques

[0 1 2 0 0 2 2 1 2 2 2 1 0 1 0 2 0 1 1 0 1 0 2 2 2 1 0 2 0 1 2 2 1 2 1 0 0
 0 0 1 1 2 1 1 1 0 0 1 1 1 1 2 0 0 1 0 1 1 2 0 1 2 1 2 2 1 2 0 1 1 1 0 2 2
 0 0 2 0 0 1 1 1 2 0 2 1 1 0 1 1 0 0 2 0 1 2 0 1 2 2 1 1 1 2 2 2 2 1 0 0 1
 2 1 1 0 2 0 1 2 2 2 2 0 2 2 0 0 2 0 0 2 0 1 0 2 1 0]
[0 1 2 0 0 2 2 1 1 2 2 1 0 1 0 2 0 1 1 0 1 0 2 2 2 2 0 1 0 1 2 2 1 2 2 0 0
 0 0 1 1 2 1 1 1 0 0 1 1 1 0 2 0 0 1 1 1 1 2 0 1 2 1 2 1 1 1 0 1 1 0 0 2 2
 0 0 2 0 0 1 1 1 2 0 2 1 1 0 1 1 0 0 2 0 1 2 0 1 1 2 1 1 1 2 1 2 2 1 0 0 1
 2 1 1 0 2 0 1 2 2 2 2 0 2 1 0 0 2 0 0 2 0 1 0 2 1 0]


In [ ]:
pipe_class.named_steps['model'].best_estimator_.feature_importances_

In [ ]:
feature_importances = pd.DataFrame(pipe_class.named_steps['model'].best_estimator_.feature_importances_,
                                   index=X.columns,
                                   columns=['importance'])

feature_importances.sort_values(by='importance', ascending=False)

In [ ]:
pipe_class.named_steps['model'].best_estimator_.feature_importances_, X.columns

In [ ]:
X.columns

In [ ]:
dot_data = StringIO()
export_graphviz(pipe_class.best_estimator_,
                out_file=dot_data,
                filled=True,
                rounded=True,
                special_characters=True,
                class_names=True
                )

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())

Image(graph.create_png())